# Introduction

This notebook is a simple interactive model of emissions at the Mischabel and Windgällen huts.

The intent is to allow us explore different scenarios and to estimate the impact of possible interventions (i.e., is a given change worthwhile?).

The model focuses on food consumption, transportation (helicopter), and, where available, possibly avoidable fuel consumption (e.g., use of the generator at Mischabelhütte). It assumes that most electricity for lighting (and heating/dehumidification in winter) comes from renewables [CC 240126].

For now the model ignores fuel used for cooking because I could not find data. This consumption is unavoidable, but it could be an important part of the denominator when measuring the overall percentage impact of planned emissions reductions.

# Data sources

The model in this notebook is based on the following data and sources.

## Hut visitation data

| Year | Hut        | Overnights | Meal Sales (k CHF) | Source               |
|------|------------|------------|--------------------|----------------------|
| 2021 | Mischabel  | 2000       | n/a                | GV 220506            |
| 2021 | Windgällen | 2700       | >300               | GV 220506            |
| 2022 | Mischabel  | 3100       | n/a                | GV 230616, CC 221026 |
| 2022 | Windgällen | 3450       | 400 (?)            | CC 221026, CC 230930 |
| 2023 | Mischabel  | 3200       | n/a                | CC 230930, CC 240126 |
| 2023 | Windgällen | 3750       | n/a                | CC 230930, CC 240126 |

## Other notes about hut operation

* Fuel use
  * 900 CHF for generator fuel at Mischabel in 2023 [CC 230930]
* Heli operations
  * ~12 heli flights/season (1/week) to Mischabel [Alicia Köster, communication with Claudine Blaser]
  * Apparently each heli flight flies only ~100 kg of fresh vegetables and meat to the hut. The limiting factor appears to be
    storage/refrigeration space in the hut.
* Vegetarian meals
  * Annina and Wisi at Windgällenhütte already serve exclusively vegetarian meals 3-4 days/week [Alicia Köster]

## Food and drink-related emissions

Data on food-related emissions across the supply chain is from [Our World in Data](https://ourworldindata.org):
* https://ourworldindata.org/grapher/food-emissions-supply-chain
* https://ourworldindata.org/grapher/ghg-per-protein-poore
* https://ourworldindata.org/grapher/ghg-kcal-poore
which is based on [Poore and Nemecek, 2018](https://doi.org/10.1126/science.aaq0216), the large meta-analysis of food systems to date.

Additional data for bottled drinks comes from [Carbon Cloud](https://carboncloud.com). The impact of different containers (kegs, bottles, etc.) is based on [Bowler et al., 2014](https://doi.org/10.1016/j.jclepro.2023.140181) and [Cimini and Moresi, 2016](https://doi.org/10.1016/j.jclepro.2015.06.063). The authors provide a range of estimates: I have chosen middle-of-the-road values. Differences in life-cycle emissions for different packaging types are (for now) not incorporated further into transportation emissions (i.e., they do not affect heli emissions estimates).

## Transportation emissions

Data on helicopter performance comes from [Air Zermatt](https://air-zermatt.ch) and the Eurocopter [technical data sheet](https://www.aviatorsdatabase.com/wp-content/uploads/2013/07/AS350-B3.pdf)


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# High-level emission areas for a hut
containers = {
    'Beer bottle': {
        'weight_kg': 0.3,  # Weight in kg
        'capacity_l': 0.5,  # Capacity in liters
        'emission_factor_kg_co2_per_kg': 0.5  # CO2 emissions per kg of container
    },
    'Wine bottle': {
        'weight_kg': 0.5,
        'capacity_l': 0.75,
        'emission_factor_kg_co2_per_kg': 0.6
    },
    'Keg': {
        'weight_kg': 10,
        'capacity_l': 30,
        'emission_factor_kg_co2_per_kg': 0.4
    }
}
# kg CO2eq emissions per kg of food
food_emissions_per_kg = {
    "beef": 99,
    "lamb": 40,
    "chicken": 10,
    "pork": 12,
    "cheese": 24,
    "grains": 1.4,
    "potatoes": 0.5,
    "pulses": 2,
    "tofu": 3   
}
meal_emissions = {
    # Vegetarian meal: 100g tofu, 100g pulses, 100g grains, 100g potatoes, 50g cheese
    "vegetarian": food_emissions_per_kg["tofu"] * 0.1 + food_emissions_per_kg["pulses"] * 0.1 + food_emissions_per_kg["grains"] * 0.1 + food_emissions_per_kg["potatoes"] * 0.1 + food_emissions_per_kg["cheese"] * 0.05,
    # Beef meal: 100g beef, 100g potatoes, 100g grains, 50g cheese
    "beef": food_emissions_per_kg["beef"] * 0.1 + food_emissions_per_kg["potatoes"] * 0.1 + food_emissions_per_kg["grains"] * 0.1 + food_emissions_per_kg["cheese"] * 0.05,
    # Lamb meal: 100g lamb, 100g potatoes, 100g grains, 50g cheese
    "lamb": food_emissions_per_kg["lamb"] * 0.1 + food_emissions_per_kg["potatoes"] * 0.1 + food_emissions_per_kg["grains"] * 0.1 + food_emissions_per_kg["cheese"] * 0.05,
    # Pork meal: 100g pork, 100g potatoes, 100g grains, 50g cheese
    "pork": food_emissions_per_kg["pork"] * 0.1 + food_emissions_per_kg["potatoes"] * 0.1 + food_emissions_per_kg["grains"] * 0.1 + food_emissions_per_kg["cheese"] * 0.05,
    # Chicken meal: 100g chicken, 100g potatoes, 100g grains, 50g cheese
    "chicken": food_emissions_per_kg["chicken"] * 0.1 + food_emissions_per_kg["potatoes"] * 0.1 + food_emissions_per_kg["grains"] * 0.1 + food_emissions_per_kg["cheese"] * 0.05
}
visitation = {
    "visitor_nights_per_season": 1000,
    "fraction_vegetarian": 0.2,
    "beers_per_visitor_night": 1,
}
heating = {
    "energy_type": "propane",
    "fuel_per_season_l": 1000,
    "emission_factor_kg_co2_per_l": 1.5
}
heli_transport = {
    'num_trips': 12,  # Number of trips per season
    'fuel_burn_rate_kg_per_h': 150,
    'flight_duration_h': 0.3,  # Flight duration per round trip
    'emission_factor_kg_co2_per_kg': 3.16 # CO2 emissions per kg of kerosene
}

def single_meal_emissions(food_emissions_per_kg, visitation):
    total_meal_emissions = 0
    for food, emissions_per_kg in food_emissions_per_kg.items():
        if food in ["beef", "lamb", "chicken", "pork"]:
            total_meal_emissions += emissions_per_kg * 0.2 * 0.125
        else:
            total_meal_emissions += emissions_per_kg * 0.8 * 0.125
    return total_meal_emissions

def heli_emissions():
    return heli_transport['num_trips'] * heli_transport['fuel_burn_rate_kg_per_h'] * heli_transport['flight_duration_h'] * heli_transport['emission_factor_kg_co2_per_kg']

""" # Function to calculate food supply chain emissions
def calculate_food_emissions(params):
    return params['total_weight_kg'] * params['avg_distance_km'] * params['emission_factor_kg_co2_per_km_per_kg']

# Function to calculate drink supply chain emissions
def calculate_drink_emissions(params):
    return params['total_weight_kg'] * params['avg_distance_km'] * params['emission_factor_kg_co2_per_km_per_kg']

# Function to calculate heating emissions
def calculate_heating_emissions(params):
    return params['total_energy_consumed_liters'] * params['emission_factor_kg_co2_per_liter']
 """
def total_emissions():
    e = 0
    e += heli_emissions()
    return e

e = total_emissions()
print(f"Total CO2 Emissions: {e:.2f} kg CO2")


Total CO2 Emissions: 1706.40 kg CO2


In [3]:
from ipywidgets import interact

# Interactive function to calculate total emissions based on user input
def interactive_emissions(num_trips=12):
    heli_transport['num_trips'] = num_trips    
    e = total_emissions()
    print(f"Total CO2 Emissions: {e:.2f} kg CO2")

# Create sliders for user interaction
interact(interactive_emissions, num_trips=(1, 20))


interactive(children=(IntSlider(value=12, description='num_trips', max=20, min=1), Output()), _dom_classes=('w…

<function __main__.interactive_emissions(num_trips=12)>